In [36]:
# Core dependencies
!pip install langchain langchain-community langchain-ollama langchain-chroma sentence-transformers chromadb fastapi uvicorn streamlit pypdf


In [37]:
!pip install pypdf


In [38]:
import tkinter
tkinter._test()


In [ ]:
import tkinter as tk
from tkinter import scrolledtext
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
import os

# ---- SETTINGS ----
CHATBOT_NAME = "InfortechGPT"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 50
TOP_K = 3
DOCS_DIR = "docs"

# ---- STEP 0: Initialize embedding model ----
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# ---- STEP 0a: Load & split documents if updating vectorstore ----
if not os.path.exists("chroma_db") or len(os.listdir("chroma_db")) == 0:
    print(f"Creating new vectorstore with chunk_size={CHUNK_SIZE}...")
    loader = DirectoryLoader(DOCS_DIR, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    docs_chunks = text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(docs_chunks, embedding_model, persist_directory="chroma_db")
    vectorstore.persist()
    print("Vectorstore created and persisted.")
else:
    vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)

# ---- STEP 0b: Create retriever ----
retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

# ---- STEP 1: Initialize LLM ----
llm = ChatOllama(model="gemma3:1b")

# ---- STEP 2: Define prompt template ----
prompt = ChatPromptTemplate.from_template(
    f"{CHATBOT_NAME} is a helpful AI assistant.\nAnswer the following question based on the provided documents:\n{{context}}\n\nQuestion: {{input}}"
)

# ---- STEP 3: Create QA and RAG chains ----
qa_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

# ---- GUI ----
def ask_question():
    user_query = input_box.get().strip()
    if not user_query:
        return
    input_box.delete(0, tk.END)
    result_text.insert(tk.END, f"{CHATBOT_NAME} is thinking...\n", "thinking")
    root.update()

    try:
        result = rag_chain.invoke({"input": user_query})
        answer = result.get("answer", "No answer found.")
        result_text.insert(tk.END, f"Q: {user_query}\n{CHATBOT_NAME}: {answer}\n\n", "answer")
        result_text.see(tk.END)
    except Exception as e:
        result_text.insert(tk.END, f"Error: {str(e)}\n\n", "error")

# ---- GUI Setup ----
root = tk.Tk()
root.title(CHATBOT_NAME)
root.geometry("750x550")
root.configure(bg="#1e1e2f")  # Dark background

# Title
tk.Label(root, text=CHATBOT_NAME, bg="#1e1e2f", fg="#f1f1f1", font=("Helvetica", 20, "bold")).pack(pady=10)

# Input box
input_frame = tk.Frame(root, bg="#1e1e2f")
input_frame.pack(pady=5)
tk.Label(input_frame, text="Enter your question:", bg="#1e1e2f", fg="#f1f1f1", font=("Helvetica", 12)).pack(side=tk.LEFT, padx=5)
input_box = tk.Entry(input_frame, width=60, font=("Helvetica", 12))
input_box.pack(side=tk.LEFT, padx=5)
tk.Button(input_frame, text="Ask", command=ask_question, bg="#4CAF50", fg="white", font=("Helvetica", 12, "bold"), activebackground="#45a049").pack(side=tk.LEFT, padx=5)

# Output box
tk.Label(root, text="Answer:", bg="#1e1e2f", fg="#f1f1f1", font=("Helvetica", 12)).pack(pady=5)
result_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=90, height=25, font=("Helvetica", 12), bg="#2e2e3e", fg="#f1f1f1", insertbackground="white")
result_text.pack(pady=5)

# Text tags for color coding
result_text.tag_config("thinking", foreground="#FFD700", font=("Helvetica", 12, "italic"))
result_text.tag_config("answer", foreground="#00FF7F", font=("Helvetica", 12))
result_text.tag_config("error", foreground="#FF6347", font=("Helvetica", 12, "bold"))

root.mainloop()
